# COMP3411/9418 21T0 Assignment 1

- Lecturer: Anna Trofimova
- School of Computer Science and Engineering, UNSW Sydney
- Last Update 13th January at 18:00pm, 2021
$$
% macros
\newcommand{\indep}{\perp \!\!\!\perp}
$$

## Student:
Zhixun Ling, z5304998

### Submission
This interactive notebook contains the instructions to complete assignment 1; You should submit this notebook with the code and answers in one single file in .ipybn format with the name assignment1.ipybn. **Write your name and zID in the cell above** (to edit the markdown text double-click the cell).

There is a maximum file size cap of 5MB, so make sure your submission does not exceed this size. The submitted notebook should contain all your source code and answers. You can add new cells and use markdown text to organise and explain your implementation/answer.

Submit your files using give. On a CSE Linux machine, type the following on the command-line:

The submission deadline is **22nd January at 11.59pm, 2021.**

### Late Submission Policy 
The penalty is set at 20% per late day. This is ceiling penalty, so if your submission is marked 60/100 and it was submitted two days late, you still get 60/100. If you submit 5 days later, then the penalty is 100% and your mark will be 0.

### Plagiarism
This is an individual assignment. Remember that **all** work submitted for this assignment must be your own **individual** work and no code
sharing or copying is allowed. You may use code from the Internet only with suitable attribution
of the source in your program. **Do not use public code repositories as your code might be copied.** Keep in mind that sharing parts of assignment solutions is a form of plagiarism. All submitted assignments will be run through plagiarism detection software to detect similarities to other submissions. You should carefully read the UNSW policy on academic integrity and plagiarism. 

### **Note: if your life is falling apart and you are tempted to plagiarize as to not fail the assignment, please send me an email instead. I would rather grant you an extention than report you to the Student Conduct and Integrity Unit**.

### Description

This assignment consists of two problems presented in a puzzle game context. For the first problem, you will be implementing a path search problem along with common search algorithms and evaluating their properties. For the second problem, you will be exploring how to define and solve a constraint satisfaction problem. To implement the solutions, you have to use AIPython code that provides you with many implemented classes and examples of their use.  

When working with a Jupyter notebook, you can edit the \*.py files either in the Jupyter interface (in your browser) or with your favorite editor (e.g., PyCharm). Whenever you save a \*.py file, the notebook will reload their content directly.

The libraries that you can use (and need) in this assignment are  the following (run the cell below to import the libraries):

In [6]:
import time
import numpy as np
import sys
import copy
sys.path.insert(0, "aipython")
from aipython.searchGeneric import *
from aipython.searchProblem import *
from aipython.cspProblem import *
from aipython.cspSearch import *

*Tip: If you placed the notebook correctly it should not give you any errors.*

*If you think that there are other libraries you might need then send an email to the lecturer-in-charge to confirm.*

### Part 1 - Heuristics [5 marks]

This part of the assignment is based on a popular puzzle, Game of Fifteen, also known as 15-puzzle. The game consists of 15 numbered tiles on a four by four grid where one tile is missing. To solve the puzzle, the tiles must be moved so that they are ordered from 1 to 15. The goal state of the puzzle is defined as a multidimensional array of digits#, where 0 represents the missing tile, as follow:

In [8]:
goal = [[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]]

#### Task 1.1
Edit the code below to implement a search problem class representing the Game of Fifteen that uses heuristic based on Manhattan Distance (h1). The class must be implemented by extending the class Search\_problem. 

Manhattan Distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{M}(p1, p2) = |x1 - x2| + |y1 - y2|$

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of Search_problem_from_explicit_graph class in searchProblem.py*


In [10]:
class GameFifteenProblem(Search_problem):
    
    def __init__(self, start, goal):
        self.start_ = start
        self.goal_ = goal
        self.g_ = np.copy(self.goal_)

        
    def start_node(self):
        """Returns the start node"""
        return self.start_
    
    def is_goal(self, node):
        """Returns True if the node is the goal, otherwise False"""
        try:
            for i in range(len(self.goal_)):
                for j in range(len(self.goal_[i])):
                    if(self.goal_[i][j] != node[i][j]):
                        return False
        except:
            return False
        
        return True
    
    
    def neighbors(self, node):
        """Returns a list of the arcs for the neighbors of node, for example: 
        return [Arc(node, to_neighbor_node1, cost1), Arc(node, to_neighbor_node2, cost2)]"""
        ret = []
        
        # find the empty tile
        loc = np.argwhere(np.copy(node) == 0)
        x, y = loc[0][0], loc[0][1]
        
        if x > 0:
            nb = copy.deepcopy(node)
            nb[x][y] = nb[x-1][y]
            nb[x-1][y] = 0
            ret.append(Arc(copy.deepcopy(node), nb, 1))
            
        if x < 3:
            nb = copy.deepcopy(node)
            nb[x][y] = nb[x+1][y]
            nb[x+1][y] = 0
            ret.append(Arc(copy.deepcopy(node), nb, 1))
            
        if y > 0:
            nb = copy.deepcopy(node)
            nb[x][y] = nb[x][y-1]
            nb[x][y-1] = 0
            ret.append(Arc(copy.deepcopy(node), nb, 1))
            
        if y < 3:
            nb = copy.deepcopy(node)
            nb[x][y] = nb[x][y+1]
            nb[x][y+1] = 0
            ret.append(Arc(copy.deepcopy(node), nb, 1))
        
        return ret
    
    
    def heuristic(self, node):
        """Returns the heuristic value of the node 
        based on the Manhattan distance"""
        n = np.copy(node)
        
        dist = 0
        
        for i in range(1, 16):
            loc = np.argwhere(self.g_ == i)
            gx, gy = loc[0][0], loc[0][1]
            
            loc = np.argwhere(n == i)
            nx, ny = loc[0][0], loc[0][1]
            
            dist += abs(gx - nx) + abs(gy - ny)
        
        return dist

To validate the correctness of the problem class use the A* searcher algorithm (from searchGeneric.py) to find a solution. 

*Tip: The cost of the solution should be 9.*

In [12]:
start = [[1, 2, 3, 4], 
         [9, 5, 6, 7], 
         [10, 11, 8, 0], 
         [13, 14, 15, 12]]

puzzle = GameFifteenProblem(start, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

11 paths have been expanded and 23 paths remain in the frontier
Cost:  9


#### Task 1.2 
Implement search problem classes representing the Game of Fifteen that use heuristics based on Euclidean Distance (h2) and the number of the inversions of the permutation (h3). The classes must be implemented by extending the class GameFifteenProblem.

Euclidean distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{E}(p1, p2) =  \sqrt{(x1-x2)^{2}+ (y1-y2)^{2}}$

An inversion of a permutation $(t_{1},t_{2},...,t_{n})$ of the elements in a finite n-element set of positive integers $A=\{1,2,...,n\}$ is the pair $(t_{j},t_{k})$, where $j<k$ and $t_{j}>t_{k}$.

$\sum_{j=1}^{N} \sum_{k=j}^{N}    \{t_{j} > t_{k}: 1, otherwise: 0\}$, where $N$ is the total number of elements and $t_{i}$ is the value of i-th element.

In [13]:
class GameFifteenProblemEuclidean(GameFifteenProblem):
    def __init__(self, start, goal):
        (super().__init__(start, goal))

    def heuristic(self, node):
        """Returns the heuristic value of the node 
        based on the Euclidean distance"""
        g = np.copy(self.goal_)
        n = np.copy(node)
        
        dist = 0
        
        for i in range(1, 16):
            loc = np.argwhere(g == i)
            gx, gy = loc[0][0], loc[0][1]
            
            loc = np.argwhere(n == i)
            nx, ny = loc[0][0], loc[0][1]
            
            dist += np.sqrt((gx - nx) ** 2 + (gy - ny) ** 2)
        
        return dist

In [16]:
class GameFifteenProblemInversions(GameFifteenProblem):
    def __init__(self, start, goal):
        (super().__init__(start, goal))

    def heuristic(self, node):
        """Returns the heuristic value of the node 
        based on the sum of the inversion number of a permutation"""
        n = np.copy(node).flatten()
        
        dist = 0
        
        for j in range(0, 16):
            if n[j] == 0:
                continue
            for k in range(j + 1, 16):
                if n[k] != 0 and n[j] > n[k]:
                    dist += 1
                    
        return dist

#### Task 1.3
Run A* Search algorithm with every heuristic for the following three start states:

In [17]:
def test_h(puzzle, title):
    searcher = AStarSearcher(puzzle)
    solution = searcher.search()
    print("[{}] h-value: {}".format(title, puzzle.heuristic(puzzle.start_node())))
    print("[{}] cost: {}".format(title, solution.cost))

# optimal path cost: 14
start14 = [[1, 2, 8, 3],
           [5, 6, 7, 4],
           [9, 15, 14, 11],
           [13, 10, 12, 0]]

test_h(GameFifteenProblem(start14, goal), "start14 Manhattan")
test_h(GameFifteenProblemEuclidean(start14, goal), "start14 Euclidean")
test_h(GameFifteenProblemInversions(start14, goal), "start14 Inversions")

# optimal path cost: 17
start17 = [[1, 3, 6, 4],
           [5, 2, 8, 14],
           [9, 15, 7, 0],
           [13, 10, 12, 11]]

test_h(GameFifteenProblem(start17, goal), "start17 Manhattan")
test_h(GameFifteenProblemEuclidean(start17, goal), "start17 Euclidean")
test_h(GameFifteenProblemInversions(start17, goal), "start17 Inversions")

# optimal path cost: 23
start23 = [[1, 3, 6, 4],
           [5, 8, 15, 14],
           [9, 2, 7, 0],
           [13, 10, 12, 11]]

test_h(GameFifteenProblem(start23, goal), "start23 Manhattan")
test_h(GameFifteenProblemEuclidean(start23, goal), "start23 Euclidean")
test_h(GameFifteenProblemInversions(start23, goal), "start23 Inversions")

45 paths have been expanded and 85 paths remain in the frontier
[start14 Manhattan] h-value: 12
[start14 Manhattan] cost: 14
196 paths have been expanded and 390 paths remain in the frontier
[start14 Euclidean] h-value: 9.656854249492381
[start14 Euclidean] cost: 14
30 paths have been expanded and 62 paths remain in the frontier
[start14 Inversions] h-value: 20
[start14 Inversions] cost: 14
28 paths have been expanded and 61 paths remain in the frontier
[start17 Manhattan] h-value: 17
[start17 Manhattan] cost: 17
309 paths have been expanded and 704 paths remain in the frontier
[start17 Euclidean] h-value: 13.485281374238571
[start17 Euclidean] cost: 17
1029 paths have been expanded and 2422 paths remain in the frontier
[start17 Inversions] h-value: 23
[start17 Inversions] cost: 17
617 paths have been expanded and 1309 paths remain in the frontier
[start23 Manhattan] h-value: 19
[start23 Manhattan] cost: 23
10083 paths have been expanded and 22192 paths remain in the frontier
[start23 

In each case record in the tables below the heuristic values for the start states, the number of the expanded nodes, and the costs of the solutions.

| Heuristic | h-value: start14 | h-value: start17 | h-value: start23 |
|-----------|------------------|------------------|------------------|
|Manhattan  | 12                | 17                | 19                | 
|Euclidean  | 9.656854249492381                | 13.485281374238571                | 16.071067811865476                | 
|Inversions | 30                | 23                | 29                | 

| Heuristic | expanded: start14 | expanded: start17 | expanded: start23 |
|-----------|-------------------|-------------------|-------------------|
|Manhattan  | 45                | 28                 | 617                 | 
|Euclidean  | 196                 | 309                 | 10083                 |  
|Inversions | 30                 | 1029                 | 87774                 |  

| Heuristic | path cost: start14 | path cost: start17 | path cost: start23 |
|-----------|--------------------|--------------------|--------------------|
|Manhattan  | 14                  | 17                  | 23                  | 
|Euclidean  | 14                 | 17                  | 23                  |
|Inversions | 14                  | 17                  | 27                  |

Comment on the performance of the A* search algorithm with each heuristic based on the results in the tables. Explain which heuristic is better/worse and why.

### Part 2:  Search algorithms [7 marks]

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of AStarSearcher and  Searcher classes in searchGeneric.py*

*Tip: To initialize the frontier think of the type of structure used by the algorithm to store generated nodes. If it uses a priority queue use FrontierPQ from serachGeneric.py*

#### Task 2.1
Implement a class that performs Breadth First Search by extending class Searcher.

In [18]:
class BreadthFirstSearcher(Searcher):
    
    def __init__(self,  problem):
        super().__init__(problem)

    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier_ = []

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return len(self.frontier_) == 0

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier_.append(path)
    
    """returns (next) path from the problem's start node
        to a goal node. """
    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        visited = set()
        start_t = time.time()
        
        while not self.empty_frontier():
            if len(self.frontier_) >= 1000000:
                print("Over Mem")
                return None
            
            if time.time() - start_t > 300:
                print("Over Time")
                return None
                
            path = self.frontier_.pop(0)
            
            self.num_expanded += 1
            
            sign = tuple(tuple(i) for i in path.end())
            if sign in visited:
                continue
            visited.add(sign)
            
            if self.problem.is_goal(path.end()):    # solution found
                print("Expanded {}, Remained: {}".format(self.num_expanded, len(self.frontier_)))
                self.solution = path   # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                for arc in reversed(list(neighs)):
                    np = Path(path, arc)
                    sign = tuple(tuple(i) for i in np.end())
                    if sign not in visited:
                        self.add_to_frontier( np )
        
        print("No more solution. explored".format(self.num_expaned))
        return None

    
def test_s(searcher, title):
    print("-" * 60 + "\n" + title)
    
    solution = searcher.search()    
    if solution is not None:
        print("Cost: {}".format(solution.cost))
        
    
GFP = GameFifteenProblem
start1 = [[1, 2, 3, 4],
           [5, 6, 7, 8],
           [9, 10, 11, 12],
           [13, 14, 0, 15]]

# optimal path cost: 10
start10 = [[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]]

test_s(BreadthFirstSearcher(GFP(start1, goal)), "start1 BFS")
test_s(BreadthFirstSearcher(GFP(start10, goal)), "start10 BFS")

------------------------------------------------------------
start1 BFS
Expanded 2, Remained: 2
Cost: 1
------------------------------------------------------------
start10 BFS
Expanded 6142, Remained: 6253
Cost: 10


#### Task 2.2

Implement a class that performs Iterative Deepening Search by extending class Searcher.

In [19]:
class IterativeDeepeningSearcher(Searcher):
    def __init__(self, problem):
        super().__init__(problem)
        
        self.visited_ = None
        self.start_t_ = None
        self.path_ = None

    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier_ = []

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return len(self.frontier_) == 0

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier_.append(path)
    
    def search(self):
        self.start_t_ = time.time()
        self.path_ = self.frontier_.pop(0)
        
        for limit in range(1, 1000000):
            self.visited_ = set()

            ret = self.dfs_(0, limit)
            
            if ret is not None:
                print("Expanded {}, Remained: {}".format(self.num_expanded, len(self.frontier_)))
                self.solution = ret
                return ret
        
            if time.time() - self.start_t_ > 300:
                print("Time")
                return None
        
        # exceed 1000000 memory limit
        print("Memory")
        return None
        
    
    def dfs_(self, depth, limit):
        if self.problem.is_goal(self.path_.end()):
            print("Found solution")
            return self.path_
        
        if depth >= limit:
            return None
        
        sign = (tuple(tuple(i) for i in self.path_.end()), depth)
        if sign in self.visited_:
            return None
        self.visited_.add(sign)
        
        self.num_expanded += 1
        
        neighs = self.problem.neighbors(self.path_.end())
        for arc in list(neighs):
            tmp = self.path_
            
            # add new node
            self.path_ = Path(self.path_, arc)
            ret = self.dfs_(depth + 1, limit)
            
            # find solution
            if ret is not None:
                return ret
            
            if time.time() - self.start_t_ > 300:
                return None
            
            # backtracking to previous path
            self.path_ = tmp
        
        return None
            
        
def test_s(searcher, title):
    print("-" * 60 + "\n" + title)
    
    solution = searcher.search()    
    if solution is not None:
        print("Cost: {}".format(solution.cost))
        print(solution)
        
    
GFP = GameFifteenProblem
start1 = [[1, 2, 3, 4],
           [5, 6, 7, 8],
           [9, 10, 11, 12],
           [13, 14, 0, 15]]

# optimal path cost: 10
start10 = [[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]]

# optimal path cost: 24
start24 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 8],
           [9, 10, 13, 14]]

test_s(IterativeDeepeningSearcher(GFP(start1, goal)), "start1 IDS")
test_s(IterativeDeepeningSearcher(GFP(start10, goal)), "start10 IDS")
test_s(IterativeDeepeningSearcher(GFP(start24, goal)), "start24 IDS")

------------------------------------------------------------
start1 IDS
Found solution
Expanded 1, Remained: 0
Cost: 1
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]] --> [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
------------------------------------------------------------
start10 IDS
Found solution
Expanded 4078, Remained: 0
Cost: 10
[[2, 3, 7, 4], [1, 6, 11, 8], [5, 10, 0, 12], [9, 13, 14, 15]] --> [[2, 3, 7, 4], [1, 6, 0, 8], [5, 10, 11, 12], [9, 13, 14, 15]] --> [[2, 3, 0, 4], [1, 6, 7, 8], [5, 10, 11, 12], [9, 13, 14, 15]] --> [[2, 0, 3, 4], [1, 6, 7, 8], [5, 10, 11, 12], [9, 13, 14, 15]] --> [[0, 2, 3, 4], [1, 6, 7, 8], [5, 10, 11, 12], [9, 13, 14, 15]] --> [[1, 2, 3, 4], [0, 6, 7, 8], [5, 10, 11, 12], [9, 13, 14, 15]] --> [[1, 2, 3, 4], [5, 6, 7, 8], [0, 10, 11, 12], [9, 13, 14, 15]] --> [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [0, 13, 14, 15]] --> [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 0, 14, 15]] --> [[1, 2, 3, 4], [5, 6, 7,

KeyboardInterrupt: 

#### Task 2.3
Implement a class that performs Iterative Deepening A* Search by extending class Searcher.

In [ ]:
class IterativeDeepeningAStarSearcher(Searcher):
    def __init__(self, problem):
        super().__init__(problem)
        
        self.visited_ = None
        self.start_t_ = None
        self.path_ = None
        self.node_generate_ = None

    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier_ = []

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return len(self.frontier_) == 0

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier_.append(path)
    
    def search(self):
        self.start_t_ = time.time()
        self.path_ = self.frontier_.pop(0)
        self.node_generate_ = 0
        
        for limit in range(1, 100, 2):
            self.visited_ = set()
            ret = self.dfs_(limit)
            
            if ret is not None:
                print("Expanded {}, Generated: {}".format(self.num_expanded, self.node_generate_))
                self.solution = ret
                return ret
        
            if time.time() - self.start_t_ > 300:
                print("Expanded: {}, Status: {}".format(self.num_expanded, "Time"))
                return None
        
        # exceed 1000000 memory limit
        print("Memory")
        return None
        
    
    def dfs_(self, limit):
        if self.problem.is_goal(self.path_.end()):
            print("Found solution")
            return self.path_
        
        self.num_expanded += 1
        
        neighs = self.problem.neighbors(self.path_.end())
        q = FrontierPQ()
        for arc in list(neighs):
            path = Path(self.path_, arc)
            value = path.cost + self.problem.heuristic(path.end())
            
            if value <= limit:
                q.add(path, value)
                self.node_generate_ += 1
            
        while not q.empty():
            tmp = self.path_
            self.path_ = q.pop()
            
            ret = self.dfs_(limit)
            
            # find solution
            if ret is not None:
                return ret
            
            if time.time() - self.start_t_ > 300:
                return None
            
            # backtracking to previous path
            self.path_ = tmp
        
        return None
            
        
def test_s(searcher, title):
    print("-" * 60 + "\n" + title)
    
    solution = searcher.search()    
    if solution is not None:
        print("Cost: {}".format(solution.cost))
        print(solution)
        
    
GFP = GameFifteenProblem
start1 = [[1, 2, 3, 4],
           [5, 6, 7, 8],
           [9, 10, 11, 12],
           [13, 14, 0, 15]]

# optimal path cost: 10
start10 = [[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]]

# optimal path cost: 24
start24 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 8],
           [9, 10, 13, 14]]

print(test_s(IterativeDeepeningAStarSearcher(GFP(start1, goal)), "start1 IDSA*"))
test_s(IterativeDeepeningAStarSearcher(GFP(start10, goal)), "start10 IDSA*")
test_s(IterativeDeepeningAStarSearcher(GFP(start24, goal)), "start24 IDSA*")

#### Task 2.4
Implement a class that performs Uniform Cost Search by extending class Searcher.

In [ ]:
class UniformCostSearcher(Searcher):
    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return self.frontier.empty()

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier.add(path, path.cost)
        if len(self.frontier) > 1000000:
            print("Memory")
            raise

#### Task 2.5

Run Breadth First Search (BFS), Iterative Deepenining Search (IDS), Iterative Deepening A* Search (IDA\*S), and Uniform Cost Search (UCS) algorithms on each of the 5 start states:

In [ ]:
# optimal path cost: 10
start10 = [[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]]

# optimal path cost: 24
start24 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 8],
           [9, 10, 13, 14]]

# optimal path cost: 30
start30 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 14],
           [9, 10, 8, 13]]

# optimal path cost: 36
start36 = [[7, 11, 12, 4],
           [2, 3, 14, 1],
           [6, 5, 13, 8],
           [9, 10, 15, 0]]

# optimal path cost: 41
start41 = [[7, 11, 12, 4],
           [2, 3, 8, 14],
           [10, 0, 5, 1],
           [6, 9, 13, 15]]


def test_s(searcher, title):
    print("-" * 60 + "\n" + title)
    solution = searcher.search()
    if solution is not None:
        print("[{}] cost: {}".format(title, solution.cost))
    
GFP = GameFifteenProblem

# your code
try:
    test_s(BreadthFirstSearcher(GFP(start10, goal)), "start10 BFS")
    test_s(BreadthFirstSearcher(GFP(start24, goal)), "start24 BFS")
    test_s(BreadthFirstSearcher(GFP(start30, goal)), "start30 BFS")
    test_s(BreadthFirstSearcher(GFP(start36, goal)), "start36 BFS")
    test_s(BreadthFirstSearcher(GFP(start41, goal)), "start41 BFS")
except:
    print("Continue to next algorithm")

try:
    test_s(IterativeDeepeningSearcher(GFP(start10, goal)), "start10 IDS")
    test_s(IterativeDeepeningSearcher(GFP(start24, goal)), "start24 IDS")
    test_s(IterativeDeepeningSearcher(GFP(start30, goal)), "start30 IDS")
    test_s(IterativeDeepeningSearcher(GFP(start36, goal)), "start36 IDS")
    test_s(IterativeDeepeningSearcher(GFP(start41, goal)), "start41 IDS")
except:
    print("Continue to next algorithm")

try:
    test_s(IterativeDeepeningAStarSearcher(GFP(start10, goal)), "start10 IDA*")
    test_s(IterativeDeepeningAStarSearcher(GFP(start24, goal)), "start24 IDA*")
    test_s(IterativeDeepeningAStarSearcher(GFP(start30, goal)), "start30 IDA*")
    test_s(IterativeDeepeningAStarSearcher(GFP(start36, goal)), "start36 IDA*")
    test_s(IterativeDeepeningAStarSearcher(GFP(start41, goal)), "start41 IDA*")
except:
    print("Continue to next algorithm")

try:
    test_s(UniformCostSearcher(GFP(start10, goal)), "start10 UCS")
    test_s(UniformCostSearcher(GFP(start24, goal)), "start24 UCS")
    test_s(UniformCostSearcher(GFP(start30, goal)), "start30 UCS")
    test_s(UniformCostSearcher(GFP(start36, goal)), "start36 UCS")
    test_s(UniformCostSearcher(GFP(start41, goal)), "start41 UCS")
except:
    print("Continue to next algorithm")

try:
    test_s(AStarSearcher(GFP(start10, goal)), "start10 A*")
    test_s(AStarSearcher(GFP(start24, goal)), "start24 A*")
    test_s(AStarSearcher(GFP(start30, goal)), "start30 A*")
    test_s(AStarSearcher(GFP(start36, goal)), "start36 A*")
    test_s(AStarSearcher(GFP(start41, goal)), "start41 A*")
except:
    print("Done")

In each case, record in the table below the number of nodes generated during the search. If the algorithm runs out of memory or the number of nodes in the frontier exceeds 1 million items, just write “Mem” in your table. If the code runs for five minutes without producing output, terminate the process and write “Time” in your table.

*Tip: To edit the table double click the cell below.*

| Algorithm | 10      | 24      | 30      | 36      | 41      |
|-----------|---------|---------|---------|---------|---------|
|BFS        | 12395       | Mem       | Mem       | Mem       | Mem       |
|A*         | 31       |  3461      | 3975       | 233442       | Time       |
|IDS        | 4078       | Time       | Time       | Time       | Time       |
|IDA\*S     | 15       | 1171       | 11323       | 96005       | Time       |
|UCS        | 648636       | Mem       | Mem       | Mem       | Mem       |

Discuss the time and space efficiency of these four algorithms, comment on the results in the table.

### Part 3: Deceptive Starting States [3 marks]

#### Task 3.1 

Run IDA* on the starting states below and report the number of the expanded nodes.

In [ ]:
start37_1 = [[7, 11, 12, 4],
             [2, 3, 14, 1],
             [6, 5, 13, 0],
             [9, 10, 15, 8]]

start37_2 = [[7, 11, 12, 4],
             [2, 3, 8, 14],
             [6, 0, 1, 15],
             [9, 5, 10, 13]]
# your code

def test_as(searcher, title):
    print("-" * 60 + "\n" + title)
    solution = searcher.search()
    if solution is not None:
        print("[{}] cost: {}".format(title, solution.cost))
        
test_as(IterativeDeepeningAStarSearcher(GFP(start37_1, goal)), "start37_1 IDA*")
test_as(IterativeDeepeningAStarSearcher(GFP(start37_2, goal)), "start37_2 IDA*")

#### Task 3.2

Explain why there is a difference between the number of the expanded nodes for these starting states if their costs of the optimal paths are the same.

### Part 4:  Constraint Satisfaction Problem [5 marks]

This part of the assignment is based on another puzzle called Sudoku. The game consists of 9x9 grid with a few cells filled in with digits. The objective of the game is to fill the remaining cells so that each column, each row, and each of the nine 3×3 sub-grids contain all of the digits from 1 to 9.

The start state of the puzzle is defined as a multidimensional array of numbers, where 0 represents empty cells, for example:

In [ ]:
grid = [[9, 5, 0, 8, 2, 7, 3, 0, 0],
        [0, 8, 0, 1, 4, 0, 0, 5, 0],
        [0, 1, 0, 5, 9, 0, 0, 0, 0],
        [8, 3, 0, 0, 0, 0, 0, 7, 5],
        [1, 6, 9, 7, 5, 2, 4, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 6, 0],
        [0, 9, 1, 0, 6, 0, 8, 4, 0],
        [7, 0, 8, 0, 3, 1, 0, 0, 6],
        [6, 2, 0, 4, 7, 8, 0, 9, 0]]

*Tip: If you have problems with understanding how to implement constrains take a look at cspExamples.py and cspExamplesQueens.py*

#### Task 4.1

Write the code that defines constraint(s) and a function grid_to_csp that returns a CSP for a Sudoku puzzle:

In [ ]:
# define constraint function(s)

# function that returns a csp
def grid_to_csp(grid):
    domains = list(range(9))
    
    constraints = [
        
    ]
    
    return CSP(domains, constraints)

# csp
csp = grid_to_csp(grid)
print(csp)

#### Task 4.2

Write the code that solves Sudoku csp using a search algorithms of your choice, print the solution.

In [ ]:
# your code
def get_next(g):
    loc = np.argwhere(np.copy(g) == 0)
    if len(loc) > 0:
        return loc[0][0], loc[0][1]
    else:
        return -1, -1

def validate(g, i, j, assignment):
    row_validation = all([assignment != g[i][c] for c in range(0, 9)])
    col_validation = all([assignment != g[r][j] for r in range(0, 9)])

    if not all([row_validation, col_validation]):
        return False

    ulx = 3 * (i // 3) 
    uly = 3 * (j // 3)
    block_validation = all([assignment != g[r][c] for r in range(ulx, ulx + 3) for c in range(uly, uly + 3)])
    
    return block_validation


def dfs(g, i, j):
    i, j = get_next(g)

    if i == -1:
        return True

    for assignment in range(1,10):
        if not validate(g, i, j, assignment):
            continue
        
        g[i][j] = assignment

        if dfs(g, i, j):
            return True
        
        g[i][j] = 0
    
    return False


def solve(g):
    return dfs(g, *get_next(g))

if solve(grid):
    print(np.array(grid))
else:
    print("Cannot find a solution")

#### Task 4.3

Descibe what do nodes represent in the search tree. Explain your choice of the search algorithm and compare it with any other search algorithm implemented in this assignemnt. 